# Converting jsons file from the sketchy website

In [11]:
import pandas as pd
import json

In [12]:
import pandas as pd

weather_data = pd.DataFrame(columns=["date"])
for month in range(1,12+1):
    with open(f'data/jsons_weather_monthly/{str(month).zfill(2)}.json') as f:
        data = json.load(f)

    for daily in data["data"]["weather"]:
        hourly_data = []
        for hourly in daily["hourly"]:
            hourly_data.append(pd.DataFrame.from_dict(hourly))
        daily_data = pd.concat(hourly_data)
        daily_data["date"] = daily["date"]
        weather_data = pd.concat([weather_data, daily_data])

weather_data.to_csv("data/weather_hourly.csv")

In [13]:
weather_data

,date,time,tempC,tempF,windspeedMiles,windspeedKmph,winddirDegree,winddir16Point,weatherCode,weatherIconUrl,...,HeatIndexF,DewPointC,DewPointF,WindChillC,WindChillF,WindGustMiles,WindGustKmph,FeelsLikeC,FeelsLikeF,uvIndex
0,2022-01-01,0,7,44,4,6,337,NNW,113,{'value': 'https://cdn.worldweatheronline.com/...,...,44,-3,27,6,43,12,19,6,43,1
0,2022-01-01,100,6,43,4,6,337,NNW,113,{'value': 'https://cdn.worldweatheronline.com/...,...,43,-4,24,5,41,13,21,5,41,1
0,2022-01-01,200,6,42,4,6,338,NNW,113,{'value': 'https://cdn.worldweatheronline.com/...,...,42,-6,22,4,40,13,21,4,40,1
0,2022-01-01,300,5,41,4,6,339,NNW,113,{'value': 'https://cdn.worldweatheronline.com/...,...,41,-7,19,4,39,13,21,4,39,1
0,2022-01-01,400,4,40,4,6,342,NNW,113,{'value': 'https://cdn.worldweatheronline.com/...,...,40,-8,17,3,37,13,21,3,37,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2022-12-31,1900,8,46,4,7,226,SW,113,{'value': 'https://cdn.worldweatheronline.com/...,...,46,3,38,7,44,16,26,7,44,1
0,2022-12-31,2000,9,48,5,7,223,SW,113,{'value': 'https://cdn.worldweatheronline.com/...,...,48,2,36,8,46,16,25,8,46,1
0,2022-12-31,2100,8,47,5,8,219,SW,113,{'value': 'https://cdn.worldweatheronline.com/...,...,47,1,34,7,45,16,26,7,45,1
0,2022-12-31,2200,8,46,5,8,227,SW,113,{'value': 'https://cdn.worldweatheronline.com/...,...,46,-3,26,7,44,18,30,7,44,1


### filter interesting columns

In [14]:
weather_data = weather_data[['date', 'time', 'tempC','windspeedKmph','weatherCode','precipMM','humidity','pressure','cloudcover','WindChillC','WindGustKmph','FeelsLikeC','uvIndex']]

In [15]:
weather_data

,date,time,tempC,windspeedKmph,weatherCode,precipMM,humidity,pressure,cloudcover,WindChillC,WindGustKmph,FeelsLikeC,uvIndex
0,2022-01-01,0,7,6,113,0.0,66,1032,0,6,19,6,1
0,2022-01-01,100,6,6,113,0.0,64,1032,0,5,21,5,1
0,2022-01-01,200,6,6,113,0.0,62,1032,0,4,21,4,1
0,2022-01-01,300,5,6,113,0.0,60,1032,0,4,21,4,1
0,2022-01-01,400,4,6,113,0.0,58,1032,0,3,21,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2022-12-31,1900,8,7,113,0.0,72,1026,3,7,26,7,1
0,2022-12-31,2000,9,7,113,0.0,69,1026,2,8,25,8,1
0,2022-12-31,2100,8,8,113,0.0,66,1027,0,7,26,7,1
0,2022-12-31,2200,8,8,113,0.0,54,1027,2,7,30,7,1


## Combine the datasets in one csv file

In [16]:
import csv
import datetime
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Read weather data
weather_data = {}

with open("data/weather_hourly.csv", "r") as f:
    reader = csv.DictReader(f)
    for row in reader:
        date = row["date"]
        time = row["time"]
        hour = int(time) // 100
        dt = datetime.datetime.strptime(date, "%Y-%m-%d").replace(hour=hour)
        weather_data[dt] = row
"""
# Read and encode occupation data
occupation_df = pd.read_csv("data/ocupacio_enginyeria_2022.csv")
columns_to_encode = ['Espai', 'Estudi', 'Activitat', 'Modalitat docencia']

#for col in columns_to_encode:
#    label_encoder = LabelEncoder()
#    occupation_df[col] = label_encoder.fit_transform(occupation_df[col].astype(str))

occupation_data = {}

for _, row in occupation_df.iterrows():
    if not row["Data inicial"].strip():  # Check if the date string is empty
        continue
    start_date = datetime.datetime.strptime(row["Data inicial"], "%d/%m/%Y")
    start_hour = int(row["Hora inicial"].split(":")[0])
    start_dt = start_date.replace(hour=start_hour)
    occupation_data[start_dt] = row.to_dict()

"""
# Read target data
target_data = {}

with open("data/Consum horari electricitat Enginyeries 2022.csv", "r") as f:
    reader = csv.DictReader(f, delimiter=";")
    for row in reader:
        date = datetime.datetime.strptime(row["Date"], "%d/%m/%Y")
        hour = int(row["Hour"].split(":")[0])
        dt = date.replace(hour=hour)
        target_data[dt] = row

# Merge the data
merged_data = []
for dt in sorted(weather_data.keys()):
    merged_row = weather_data[dt]
    """
    if dt in occupation_data:
        merged_row.update(occupation_data[dt])
    """
    if dt in target_data:
        merged_row.update(target_data[dt])

    merged_data.append(merged_row)

# Save the merged data to a CSV file
with open("data/merged_data.csv", "w", newline="") as f:
    fieldnames = set()
    for row in merged_data:
        fieldnames.update(row.keys())
    fieldnames = list(fieldnames)

    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for row in merged_data:
        writer.writerow(row)


In [17]:
merged_df = pd.read_csv("data/merged_data.csv")

In [18]:

merged_df = pd.read_csv("data/merged_data.csv")

#columns_to_drop = ['Unnamed: 0', 'Data inicial', 'Data final', 'Hora inicial', 'Hora final', 'time', 'weatherIconUrl', 'winddir16Point', 'Date', 'Hour', 'Observacions', 'tempF', 'WindChillF', 'HeatIndexF', 'FeelsLikeF', 'DewPointF', 'WindGustMiles', 'visibilityMiles', 'pressureInches', 'precipInches']
columns_to_drop = ['Unnamed: 0', 'weatherIconUrl', 'winddir16Point', 'Date', 'Hour', 'tempF', 'WindChillF', 'weatherDesc','HeatIndexF', 'FeelsLikeF', 'DewPointF', 'WindGustMiles', 'visibilityMiles', 'pressureInches', 'precipInches']
merged_df = merged_df.drop(columns=columns_to_drop)
merged_df["time"] = merged_df["time"]/100
# Save the updated DataFrame to a new CSV file
coloms_with_coma_decimal = ["Q-Enginyeria (Cos Central) [kWh] [Q-Enginyeria]","Q-Enginyeria (Espina 4) [kWh] [Q-Enginyeria]","Q-Enginyeria (Química) [kWh] [Q-Enginyeria]"]
for col in coloms_with_coma_decimal:
    changed = []
    for i,val in enumerate(merged_df[col]):
        if val == val:
            changed.append(val.replace(",","."))
        else: #we got a nan. interpolate by having previous value
            changed.append(changed[-1])
    
    merged_df[col] = changed

merged_df.to_csv("data/updated_merged_data.csv", index=False)


In [19]:
merged_df = pd.read_csv("data/updated_merged_data.csv")

In [20]:
merged_df.columns

Index(['winddirDegree', 'WindGustKmph', 'visibility', 'DewPointC',
       'Q-Enginyeria (Espina 4) [kWh] [Q-Enginyeria]',
       'Q-Enginyeria (Química) [kWh] [Q-Enginyeria]', 'uvIndex', 'HeatIndexC',
       'Q-Enginyeria (Cos Central) [kWh] [Q-Enginyeria]', 'precipMM',
       'windspeedMiles', 'weatherCode', 'tempC', 'cloudcover', 'windspeedKmph',
       'date', 'time', 'pressure', 'FeelsLikeC', 'WindChillC', 'humidity'],
      dtype='object')

Note: In the future we could play with [observations, Estudi,Modalitat docencia] columns and NLP to get some representation of the activity being done and its impact

In [21]:
occupation_data = pd.read_csv("data/ocupacio_enginyeria_2022.csv")

# Creating and embedding for the title of activities

In [22]:
import torch

The problem we have is that we have too many activities to create a one hot vector of each of them.
Second problem we have is that if we wanted to add a new activity this would break the model

In [23]:
#All activities we have.
occupation_data

,Data inicial,Hora inicial,Data final,Hora final,Total hores,Espai,Estudi,Activitat,Alumnes matriculats,Modalitat docencia,Observacions
0,04/01/2022,15:00,04/01/2022,18:00,3,Q1/0007,NaN,Classe presencial MUEBA - M. Eugenia Suarez,0,,
1,07/01/2022,09:00,07/01/2022,11:30,2.5,Q3/1007,NaN,Examen 102712 SSD (GEST) 2 aules,0,,
2,07/01/2022,09:00,07/01/2022,11:30,2.5,Q1/1003,NaN,Examen 104525 Intro.ciutat conemporània (GCIS)...,0,,
3,07/01/2022,09:00,07/01/2022,11:30,2.5,Q3/1011,NaN,Examen 102712 SSD (GEST) 2 aules,0,,
4,07/01/2022,09:00,07/01/2022,11:30,2.5,Q2/1013,NaN,Examen 106040 Matemàtiques (GEQ) 2 aules,0,,
...,...,...,...,...,...,...,...,...,...,...,...
42006,,,,,,Q3/0003,10064 Enginyeria de Sistemes de Telecomunicació,102712 Senyals i Sistemes Discrets - 331,Sense grup de matricula,,
42007,,,,,,Q3/0003,6084 Enginyeria de Telecomunicació / Telecommu...,102712 Senyals i Sistemes Discrets - 331,Sense grup de matricula,,
42008,,,,,,Q3/0003,10063 Enginyeria Electrònica de Telecomunicació,102712 Senyals i Sistemes Discrets - 331,Sense grup de matricula,,
42009,,,,,,Q3/0003,10088 Enginyeria Informàtica i Enginyeria de S...,102712 Senyals i Sistemes Discrets - 331,Sense grup de matricula,,


In the future it would be cool to create our own model that creates some embedding optimized for owr problem.

For now we'll use some pretrained.

In [24]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
start_token = "[CLS] "
end_token = " [SEP]"
test_txt = start_token + "This class is chaotic and boring at the same time. " + end_token
tokenized_text = tokenizer.tokenize(test_txt)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
segments_ids = [1] * len(tokenized_text)

In [26]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-multilingual-cased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()
model.to("mps")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
 

In [27]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens]).to("mps")
segments_tensors = torch.tensor([segments_ids]).to("mps")

In [28]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensors)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

In [29]:
# `token_vecs` is a tensor with sha pe [Ntokens x 768]
token_vecs = hidden_states[-2][0] #second to last hiden layer

# Calculate the average of all Ntokens token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)
sentence_embedding.shape

torch.Size([768])

In [30]:
def sentence2hidden_states(txt):
    test_txt = start_token + txt + end_token
    tokenized_text = tokenizer.tokenize(test_txt)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text) 
    segments_ids = [1] * len(tokenized_text)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens]).to("mps")
    segments_tensors = torch.tensor([segments_ids]).to("mps")
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
    
    hidden_states = outputs[2]
    return hidden_states

Here we have an embedding for each sentence of size (768)

Now we'll try to apply kmeans to kluster all titles by their semantic meaning.

In [31]:
from tqdm import tqdm 

In [32]:
vectors = {}
h_states = {}
occupations = set()
for occu in tqdm(occupation_data["Activitat"]):
    if occu not in occupations:
        h = sentence2hidden_states(occu)[-2][0].to("cpu") #get all layers first batch
        h_states[occu] =h

        # Calculate the average of all Ntokens token vectors.
        sentence_embedding = torch.mean(h, dim=0)
        vectors[occu] = sentence_embedding.numpy()
        occupations.add(occu) 

100%|██████████| 42011/42011 [00:41<00:00, 1023.64it/s]


We'll do the same but encode the classroom the activity is in the text also

In [33]:
vectors_plus = {}
h_states_plus = {}
occupations_plus = set()
for occu, espai in zip(occupation_data["Activitat"],occupation_data["Espai"]):
    if occu + " " + espai not in occupations_plus:
        h = sentence2hidden_states(occu + " " + espai)[-2][0].to("cpu") #get all layers first batch
        h_states_plus[occu + " " + espai] = h
        
        # Calculate the average of all Ntokens token vectors.
        sentence_embedding = torch.mean(h, dim=0)
        vectors_plus[occu + " " + espai] = sentence_embedding.numpy()
        occupations_plus.add(occu + " " + espai) 

Save data to pickle file for later use

In [34]:
import pickle

#bert_data = {
#    "single_ocu":{"mean_vect":vectors,"h_states":h_states},
#    "ocu_plus_space":{"mean_vect":vectors_plus,"h_states":h_states_plus}
#    }

bert_data = {
    "single_ocu":{"mean_vect":vectors},
    "ocu_plus_space":{"mean_vect":vectors_plus}
    }

# Open a file and use dump()
with open('data/bert_embedded.pkl', 'wb') as file:
      
    pickle.dump(bert_data, file)

We'll do some fast clustering to see if it makes any sense

In [35]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4, random_state=0, n_init="auto").fit(list(vectors_plus.values()))
cluster_cent = kmeans.cluster_centers_

In [36]:
cluster_map = pd.DataFrame()
cluster_map['data_index'] = list(vectors_plus.keys())
cluster_map['cluster'] = kmeans.labels_

In [37]:
cluster_map[cluster_map["cluster"]==0].value_counts()

data_index                                                           cluster
  958-03 102775-AC 1/4 Q3/1003                                       0          1
951-03 106045 Química inorg i de l'equilib (9:00-13:00) 1/2 Q4/0007  0          1
951-03 106054 Bases experim. Enginyeia química 1/2 Q4/0007           0          1
951-03 Cinètica química Q1/1007                                      0          1
951-03 Cinètica química Q1/1011                                      0          1
                                                                               ..
106043 Física - 21 Q4/0011                                           0          1
106043 Física - 21 Q4/1005                                           0          1
106043 Física - 211 Q4/0007                                          0          1
106043 Física - 211 Q4/0011                                          0          1
Eamen 104341 Enginyeria del rendiment (GED) Q1/1011                  0          1
Length: 914, dtype: i

In [38]:
cluster_map[cluster_map["cluster"]==1].value_counts()

data_index                                                                cluster
102401 Ampliació d'Operacions de Separació - 220 Q4/1009                  1          1
104356 Programació Parallela - 811 Q1/0007                               1          1
104358 Desenvolupament Daplicacions de Dades Massives - 81 Q1/0019       1          1
104357 Computació en Entorns Al Núvol - 812 Q2/0017                       1          1
104357 Computació en Entorns Al Núvol - 811 Q1/0019                       1          1
                                                                                    ..
102764 Metodologia de la Programació - 472 Q2/1005                        1          1
102764 Metodologia de la Programació - 472 Q1/0019                        1          1
102764 Metodologia de la Programació - 471 Q2/1009                        1          1
102764 Metodologia de la Programació - 471 Q1/0019                        1          1
classe del Màster de Modelització per a la Ciènc

In [39]:
cluster_map[cluster_map["cluster"]==2].value_counts()

data_index                                                               cluster
0004-01 Reunió d'equip MELISSA - Daniella Emiliani QC/0019               2          1
951-09 Docència seminaris Circulació de Fluids (GEQ) Q4/0011             2          1
951-09 Química inorgànica (recuperació classe) Q4/0007                   2          1
951-09 Química inorgànica industrial Q4/0007                             2          1
951-09 Recuperació Ciències dels materials GEQ - Fernando Novio Q3/0003  2          1
                                                                                   ..
2634-06 Prova Tesis Doctoral QC/0019                                     2          1
2634-07 Seminari de recerca DEQBA Q1/0003                                2          1
2634-08 Trucada amb una altra universitat. Laia Miranda QC/0019          2          1
2634-11 Catèring Teresa Gea QC/0015                                      2          1
assaig tesis Q1/0003                                       

In [40]:
cluster_map[cluster_map["cluster"]==3].value_counts()

data_index                                            cluster
115_19 JPO - GED Q4/0007                              3          1
Examen 104341 Enginyeria del rendiment (GED) Q1/1007  3          1
Examen 103805 FE (GEI) 2 aules Q1/1011                3          1
Examen 103805 FE (GEI) 2 aules Q1/1007                3          1
Examen 103803 Estadistica (GEI) 3 aules Q3/1011       3          1
                                                                ..
Examen 102741 GABD (GEI) 2 aules Q4/1013              3          1
Examen 102741 GABD (GEI) 2 aules Q4/1009              3          1
Examen 102741 GABD (GEI)  Q3/1011                     3          1
Examen 102740 SD (GEI) Q1/1011                        3          1
examen de practiques Q2/0007                          3          1
Length: 323, dtype: int64

We'll not sure if it makes any sense XD

We'll add to the dataset a column for the hiddent states of bert. We'll pass a text through bert if the input class doesn't exist.

# Creating dataset class

In [41]:
from torch.utils.data import Dataset, DataLoader
from sklearn.decomposition import PCA
from sklearn import preprocessing 
import torch
import numpy as np
import pandas as pd
import pickle
import datetime

In [42]:
class energyProject_dataset(Dataset):
    def __init__(self,dataset_pth,occupacio_pth,bert_embeddings_pkl_pth,pca_pkl_pth=None):
        self.df = pd.read_csv(dataset_pth)
        with (open(bert_embeddings_pkl_pth, "rb")) as openfile:
            self.bert_embeddings = pickle.load(openfile)
        self.activitivity_encoding_mode = 0
        
        #order of values in target tensor will follow this
        self.target_labels = ["Q-Enginyeria (Cos Central) [kWh] [Q-Enginyeria]","Q-Enginyeria (Espina 4) [kWh] [Q-Enginyeria]","Q-Enginyeria (Química) [kWh] [Q-Enginyeria]"]
       
        #load occupation data
        self.occupation_df = pd.read_csv(occupacio_pth)
        #we'll remove entries without date
        self.occupation_df = self.occupation_df[self.occupation_df["Data inicial"] != " "]

        #convert date string to be in the form y-m-d instead of d/m/y
        #convert hour data to datetime object so we can compare them
        for i, row in self.occupation_df.iterrows(): 
            self.occupation_df.loc[i]["Data inicial"] =  datetime.datetime.strptime(self.occupation_df.loc[i]["Data inicial"], "%d/%m/%Y").strftime("%Y-%m-%d")
            self.occupation_df.loc[i]["Hora inicial"] = datetime.datetime.strptime(self.occupation_df.loc[i]["Hora inicial"] ,"%H:%M").time()
            self.occupation_df.loc[i]["Hora final"] = datetime.datetime.strptime(self.occupation_df.loc[i]["Hora final"] ,"%H:%M").time()
        
        self.ocup_vocab = list(set(self.occupation_df["Activitat"]))
        self.espais_vocab = list(set(self.occupation_df["Espai"]))
        #Add a padding occupation
        self.espais_vocab.append("NO ESPAI")

        #Normalize all climate data to be between 0-1
        #we'll keep their scaler objects so we can transform their values back to original
        #and not loose meaning.
        self.column_scalers = {}
        columns_to_process = ['winddirDegree', 'precipMM', 'visibility', 'WindChillC',
       'humidity', 'pressure','windspeedMiles', 'uvIndex', 'DewPointC',
       'FeelsLikeC', 'tempC','weatherCode','HeatIndexC', 'WindGustKmph', 'cloudcover',
       'windspeedKmph']
     
        for col in columns_to_process:
            scaler, values = self.normalize_values(self.df[col])
            self.df[col] = values
            self.column_scalers[col] = scaler
        
        #If we want this dataset to work with batches in modes different than 0 and 1.
        #We need to know what is the maximum number of activities at the same time so we can
        #padd the samples smaller. We need the batch to have the same shape samples every time.

        #we'll use that we are iterating throught this to compute all the one_hot vectors of the encodings
        #and fit a PCA object so we can have it with dimensionality reduction.

        self.max_ocu_lenght = 0
        self.all_one_hots = []
        for i, row in self.df.iterrows():
            day2day_ocu = self.activty_class_perT(row["date"],row["time"])
            
            #find largest occupation size per hour
            day2day_ocu_l = len(day2day_ocu)
            if day2day_ocu_l > self.max_ocu_lenght:
                self.max_ocu_lenght = day2day_ocu_l
                
            if pca_pkl_pth == True: #if we have to calculate the PCA
                #compute one_hot vectors of occupation at this time
                self.all_one_hots.append(self.activity_class_one_hot(day2day_ocu))
        
        if (pca_pkl_pth == True): #nan value, recalculate and save file
            self.all_one_hots = np.array(self.all_one_hots)
            #compute PCA on all the one_hot vectors
            self.ocu_one_hot_pca = PCA(n_components=1000) #1000 size output vector (number chosen by hand)
            self.ocu_one_hot_pca.fit(self.all_one_hots)
            # Open a file and use dump()
            with open('data/pca_occupation.pkl', 'wb') as file:
                pickle.dump(self.ocu_one_hot_pca, file)
        else:
            with (open(pca_pkl_pth, "rb")) as openfile:
                self.ocu_one_hot_pca = pickle.load(openfile)
        
        ##free memory by removing unnecessary variables.
        del self.all_one_hots
        
            

    def normalize_values(self,x):
        """
        Input a list of values
        Output a sklearn scaler object and the list normalized.
        We need to keep the scaler to be able to re-scale the data back and now what value it is in reality.
        """
        to_scale = np.array(x).reshape(-1, 1) #the library needs this extra dimensions trick to interpret properly
        min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
        x_scaled = min_max_scaler.fit_transform(to_scale)
        return min_max_scaler, x_scaled
    
    def denormalize_values(self,x_n,scaler):
        """
        Given some set of values and a sklearn scaler object
        Transform back the values to their original "space"ArithmeticError
        return: set of values same shape as input
        """
        to_scale = np.array(x_n).reshape(-1, 1)
        return scaler.inverse_transform(to_scale)

    def datetime_enc(self,date, time)->torch.tensor:
        """
        Encodes incoming date and time strings as two values for each that 
        come from infering the index value on a sin function and cos function.
        Its nice beacause we encode the smoothness and circularity of the trigonometric
        functions.

        input <- (date: str, time:str)
        output -> (torch.tensor((1,5)))
        """
        
        date_obj =  datetime.datetime.strptime(date, "%Y-%m-%d")
        
        #encode year as floas
        year_enc = float(date_obj.year)/1000 # divide by 100 to have reasonable value
        
        ##
        ##Encoding: (sin, cos) value for each day month
        ##
        idx_d = date_obj.timetuple().tm_yday #day of the year number
        date_enc= [np.sin((idx_d/365) * 2*np.pi),np.cos((idx_d/365) * 2*np.pi)]  #365 days a yar +1 offset so we don't have negative value
        
        #encode time by hour in the day
        time_enc = [np.sin((time/24) * 2*np.pi),np.cos((time/24) * 2*np.pi)]  #24 hours a day. +1 offset so we don't have negative values
        
        return torch.tensor([year_enc]+date_enc+time_enc) 

    def datetime_dec(self,enc_tens):
        """
        Decodes incoming encoded date and time tensor as the two respective
        date time string values

        input <- (torch.tensor([torch.float,torch.float]))
        output -> date: str, time:str
        """
        #decode year
        year = int(enc_tens[0].item() * 1000)

        #decode date
        penc_date = np.arctan2(enc_tens[1].item(),enc_tens[2].item()) / (2*np.pi) * 365
        date = datetime.datetime(year, 1, 1) + datetime.timedelta(penc_date - 1)
        date = date.strftime("%Y-%m-%d")

        #decode time
        time = np.round((np.arctan2(enc_tens[3].item(),enc_tens[4].item()) / (2*np.pi) * 24)% 24)
        
        return date,time

    def activty_class_perT(self,date,time)->pd.DataFrame:
        """
        Returns slice of the pd Dataframe of activities active given some date and time
        """
        #filter dataset to see activities that day
        day2day_ocu = self.occupation_df[self.occupation_df["Data inicial"] == date ] 
        h = datetime.time(int(time))
        hour2hour_ocu = day2day_ocu[(day2day_ocu["Hora inicial"] <= h) & (day2day_ocu["Hora final"] > h)]
        return pd.DataFrame(hour2hour_ocu)

    def activity_class_one_hot(self,activities)->np.array:
        """ 
        returns flattenned coocurrence one-hot matrix of activities and classrooms
        """
        occurrence_matrix = np.zeros((len(self.ocup_vocab),len(self.espais_vocab)))
        for i,actv in activities.iterrows(): #iterate found activities
            ocup_idx = self.ocup_vocab.index(actv["Activitat"])
            espais_idx = self.espais_vocab.index(actv["Espai"])
            occurrence_matrix[ocup_idx,espais_idx] = 1
            
        #now flatten the occurrence matrix into a one hot vector
        one_hot = occurrence_matrix.flatten()
        return one_hot
    
    def class_one_hot(self,activitats)->torch.tensor:
        """
        Returns "one hot" encoding of activitat.
        In reality will not be a true one hot but a list of indexes
        that can later on be passed to some embedding layer
        """
        activitats["Espai"]
        one_hot_esp = [self.espais_vocab.index(key) for key in activitats["Espai"]]
        return torch.tensor(one_hot_esp)
       


    def __len__(self):
        return len(self.df) 
    
    def __getitem__(self, index):
        """
        This function will return more than one object depending on the mode it is on.
        Activity encoding mode:
            0: I.very large one-hot encoding of all the combinations of classroom and activity concatenated
                with all other features.
               II. target values
            
            1:  I. 0 but first with some PCA applied to reduce dimensionality of 
                the enormous one-hot encoding.
                II. target values
            
            2: returns four objects, 
                I.mean_encoding of activity from bert(as many as activities at time stamp), 
                II. classroom one hot encoding for each activity(as many as activities at time stamp)
                III. All other features at that time stamp
                IV. target values
            
            2.5:  returns four objects
                I. embedding tensor of activity from bert(as many as activities at time stamp), 
                II. classroom one hot encoding for each activity(as many as activities at time stamp)
                III. All other features at that time stamp
                IV. target values
            
            3: returns three objects,
                I. mean encoding of activity, classroom pair through bert (as many as activities at time stamp),
                II. All other features at that time stamp
                III. target values
            
            3.5:  returns four objects
                I. embedding tensor of activity, classroom pair through bert(as many as activities at time stamp), 
                II. classroom one hot encoding for each activity(as many as activities at time stamp)
                III. All other features at that time stamp
                IV. target values
            
            4: returns three objects, raw data thought for model handling.
                I. activities in text form paired with their classroom
                II. All other features at that time stamp
                III. target values
        """

        #get row in df for data to be evaluated:
        row = self.df.iloc[index]

        #First getting the "all other data " features tensor
        #Date-time encoding tensor
        enc_dt_tens = self.datetime_enc(row["date"],row["time"]) #date-time encoding

        #weather data tensor
        weather_tens = torch.tensor(row.drop(["date","time","Q-Enginyeria (Cos Central) [kWh] [Q-Enginyeria]","Q-Enginyeria (Espina 4) [kWh] [Q-Enginyeria]","Q-Enginyeria (Química) [kWh] [Q-Enginyeria]"]))

        #second get the target values tensor
        target_tens = torch.tensor(row[self.target_labels])
        
        #Get activities at given time and date
        activities = self.activty_class_perT(row["date"],row["time"])
        if self.activitivity_encoding_mode <= 1:
            one_hot = self.activity_class_one_hot(activities)
            
            if(self.activitivity_encoding_mode == 0):
                one_hot = torch.tensor(one_hot)
                #return values
                sample = {'x': torch.cat((enc_dt_tens,weather_tens,one_hot),axis=0), 'y': target_tens}
                return sample
            
            if(self.activitivity_encoding_mode == 1):
                one_hot = one_hot.reshape(1, -1) #create extra dimension because this counts as only one sample
                smaller_x = torch.tensor(self.ocu_one_hot_pca.transform(one_hot))[0] #get rid of extra dim
                sample = {'x': torch.cat((enc_dt_tens,weather_tens,smaller_x),axis=0), 'y': target_tens}
                return sample
        
        if self.activitivity_encoding_mode == 2:
            #get bert embeddings
            emb_activ = []
            for i,actv in activities.iterrows(): #iterate found activities   
                emb_activ.append(self.bert_embeddings["ocu_plus_space"]["mean_vect"][actv["Activitat"] + " " + actv["Espai"]])
            emb_activ = torch.tensor(np.array(emb_activ))

            #get espai one-hot
            espai_one_hot = self.class_one_hot(activities)

            #padd with occupation 0 vector so all samples are same shape and espai with ""NO ESPAI"
            if emb_activ.shape[0] < self.max_ocu_lenght: 
                emb_activ = torch.cat((emb_activ,torch.zeros((self.max_ocu_lenght-emb_activ.shape[0],768))),axis=0)
                espai_padd = pd.DataFrame(columns=["Espai"]) 
                espai_padd["Espai"] = ["NO ESPAI"]* (self.max_ocu_lenght-len(espai_one_hot))
                padd_one_hot = self.class_one_hot(espai_padd)
                espai_one_hot = torch.cat((espai_one_hot,padd_one_hot))
                
            sample = {'ocu_ber_emb': emb_activ,'espai_enc':espai_one_hot, "general_data":torch.cat((enc_dt_tens,weather_tens),axis=0), 'y': target_tens}
            return sample
        
        if self.activitivity_encoding_mode == 2.5:
            assert "NOT IMPLEMENTED YET"
            pass
            
        if self.activitivity_encoding_mode == 3:
            emb_activ = []
            for i,actv in activities.iterrows(): #iterate found activities   
                emb_activ.append(self.bert_embeddings["ocu_plus_space"]["mean_vect"][actv["Activitat"] + " " + actv["Espai"]])
            emb_activ = torch.tensor(np.array(emb_activ))

            if emb_activ.shape[0] < self.max_ocu_lenght: #padd with occupation 0 vector so all samples are same shape
                emb_activ = torch.cat((emb_activ,torch.zeros((self.max_ocu_lenght-emb_activ.shape[0],768))),axis=0)
            
            #return values
            sample = {'activ': torch.tensor(emb_acti), 'general_data':torch.cat((enc_dt_tens,weather_tens),axis=0), 'y': target_tens}
            return sample
        
        if self.activitivity_encoding_mode == 3.5:
            assert "NOT IMPLEMENTED YET"
            #errors to solve have to recalculate bert passing
            emb_acti = []
            for i,actv in activities.iterrows(): #iterate found activities   
                emb_acti.append(self.bert_embeddings["ocu_plus_space"]["h_states"][actv["Activitat"] + " " + actv["Espai"]])
                print(self.bert_embeddings["ocu_plus_space"]["h_states"][actv["Activitat"] + " " + actv["Espai"]].shape)
            emb_acti = torch.tensor(emb_acti)
            
            print(emb_acti.shape)
            #return values
            return torch.tensor(emb_acti), torch.cat((enc_dt_tens,weather_tens),axis=0) , target_tens

        if self.activitivity_encoding_mode == 4:
            assert "NOT IMPLEMENTED YET"
            pass

In [44]:
dataset = energyProject_dataset("data/updated_merged_data.csv","data/ocupacio_enginyeria_2022.csv","data/bert_embedded.pkl",True)

Save dataset as pickle so we don't have to run init every time and it is only one file to handle

In [45]:
# Open a file and use dump()
with open('data/dataset_class.pkl', 'wb') as file:
    pickle.dump(dataset, file)

# Usage instructions:

In [46]:
#import dataset class
from dataset import energyProject_dataset

In [47]:
#load dataset object file
import pickle
with (open('data/dataset_class.pkl', "rb")) as openfile:
    dataset = pickle.load(openfile)

In [48]:
#now you can create a dataloader and use it!
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=100, shuffle=False)

In [49]:
next(iter(dataloader))

{'x': tensor([[2.0220, 0.0172, 0.9999,  ..., 0.0000, 0.0000, 0.0000],
         [2.0220, 0.0172, 0.9999,  ..., 0.0000, 0.0000, 0.0000],
         [2.0220, 0.0172, 0.9999,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [2.0220, 0.0860, 0.9963,  ..., 0.0000, 0.0000, 0.0000],
         [2.0220, 0.0860, 0.9963,  ..., 0.0000, 0.0000, 0.0000],
         [2.0220, 0.0860, 0.9963,  ..., 0.0000, 0.0000, 0.0000]],
        dtype=torch.float64),
 'y': tensor([[26.5500,  1.6300, 71.3800],
         [26.7800,  1.6400, 74.7900],
         [27.0800,  1.6300, 70.6500],
         [26.7900,  1.6500, 69.7500],
         [27.2000,  1.6400, 69.6200],
         [27.0300,  1.6500, 71.2400],
         [27.4300,  1.6100, 72.7100],
         [26.4800,  1.6400, 72.4300],
         [25.1300,  1.5000, 70.7000],
         [25.9100,  1.5200, 72.1000],
         [25.1200,  1.5100, 72.3000],
         [26.0700,  1.5200, 75.0000],
         [26.5000,  1.4800, 78.6500],
         [25.6500,  1.4800, 80.8500],
         [25.5800,  1.5

By default the dataset is in 0 mode. Here are the modes:

In [50]:
dataset.__getitem__?

Signature: dataset.__getitem__(index)
Docstring:
This function will return more than one object depending on the mode it is on.
Activity encoding mode:
    0: I.very large one-hot encoding of all the combinations of classroom and activity concatenated
        with all other features.
       II. target values
    
    1:  I. 0 but first with some PCA applied to reduce dimensionality of 
        the enormous one-hot encoding.
        II. target values
    
    2: returns four objects, 
        I.mean_encoding of activity from bert(as many as activities at time stamp), 
        II. classroom one hot encoding for each activity(as many as activities at time stamp)
        III. All other features at that time stamp
        IV. target values
    
    2.5:  returns four objects
        I. embedding tensor of activity from bert(as many as activities at time stamp), 
        II. classroom one hot encoding for each activity(as many as activities at time stamp)
        III. All other features at t

To change the dataset mode just do:

In [51]:
dataset.activitivity_encoding_mode = 2 #or any value

And you don't even have to update the dataloader!

In [52]:
next(iter(dataloader))

{'ocu_ber_emb': tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0.,